This part tries to get height information from the SWOT_L2_HR_PIXC_2.0 dataset

In [53]:
# Standard library imports
from datetime import datetime
import json
from pathlib import Path
import os
import glob
import zipfile

# Third-party library imports
import geopandas as gpd
import pandas as pd
import contextily as ctx
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import netCDF4
import earthaccess

# Matplotlib inline magic command
%matplotlib inline

In [54]:
with open('config.json', 'r') as f:
    config = json.load(f)

print(config)

{'data_dir': 'data', 'output_dir': 'output', 'sampling_points_shapefile': 'res\\Reservoirs.shp', 'water_mask_pixel_cloud_dir': 'data\\Water mask pixel cloud', 'temporal_range': ['2024-02-01T00:00:00', '2024-02-29T00:00:00'], 'product_short_name': 'SWOT_L2_HR_PIXC_2.0', 'bounding_box': [36.8584632059703, -0.8194088619079362, 37.2444933070158, -0.03771035197995866]}


In [56]:
# 1. Identify and open the first NetCDF file in the pixel cloud directory
pixel_cloud_dir = Path(config['water_mask_pixel_cloud_dir'])
file_list = list(pixel_cloud_dir.glob('*.nc'))
filepath = file_list[0]  # Take the first file found

# 2. Load the pixel_cloud dataset and select the last 10,000 entries
ds = xr.open_dataset(filepath, group='pixel_cloud')

# 3. Load the bounding box from a GeoJSON file as a GeoDataFrame
bbox_path = Path(config['data_dir']) / 'bbox.geojson'
with open(bbox_path, 'r') as f:
    bbox_data = json.load(f)
bbox_gdf = gpd.GeoDataFrame.from_features(bbox_data['features'])

try:
    # 4. Clip the dataset to the bounding box 
    ds_clipped = ds.where(
        (ds.latitude >= bbox_gdf.bounds.miny[0]) &
        (ds.latitude <= bbox_gdf.bounds.maxy[0]) &
        (ds.longitude >= bbox_gdf.bounds.minx[0]) &
        (ds.longitude <= bbox_gdf.bounds.maxx[0]),
        drop=True
    )
    # 5. Convert clipped data into a GeoDataFrame (EPSG:4326)
    gdf = gpd.GeoDataFrame(
        geometry=gpd.points_from_xy(ds_clipped.longitude.values, ds_clipped.latitude.values),
        crs='EPSG:4326'
    )
    gdf.index = pd.Index(range(len(gdf)))  

except ValueError:
    print('No data in the bounding box')
    ds_clipped = ds


# 6. Populate the GeoDataFrame with relevant data columns.
# This complex part is mainly about trying to understand what may be relevant
data_columns = {
    "classification": {
        "type": int,
        "null_value": 255,
        "description": "Flags indicating water detection results.",
    },
    "classification_str": {
        "type": str,
        "null_value": "unknown",
        "value_map": {
            1: "land",
            2: "land_near_water ",
            3: "water_near_land ",
            4: "open_water ",
            5: "dark_water  ",
            6: "low_coh_water_near_land  ",
            7: "open_low_coh_water "
        },
        "description": "Flags indicating water detection results as string.",
    },
    "layover_impact": {
        "type": float,
        "null_value": 9.969209968386869e+36,
        "description": "Estimate of the height error caused by layover, which may not be reliable on a pixel by pixel basis, but may be useful to augment aggregated height uncertainties. ",
    },
    "height": {
        "type": float,
        "null_value": 9.96921e+36,
        "description": "Height of the pixel above the reference ellipsoid.",
    },
    "illumination_time": {
        "type": datetime,
        "null_value": 9.969209968386869e+36,
        "description": "Time of measurement in seconds in the UTC time scale since 1 Jan 2000 00:00:00 UTC. [tai_utc_difference] is the difference between TAI and UTC reference time (seconds) for the first measurement of the data set. If a leap second occurs within the data set, the attribute leap_second is set to the UTC time at which the leap second occurs. ",
    },

}

for column, column_data in data_columns.items():
    
    if column in ds_clipped:
        gdf[column] = ds_clipped[column].values
        try: 
            gdf[column] = gdf[column].astype(column_data['type'])
        except TypeError:
            print(f"Could not convert {column} to {column_data['type']}")
        gdf[column] = gdf[column].replace(column_data['null_value'], np.nan)
        if 'value_map' in column_data:
            gdf[column] = gdf[column].replace(column_data['value_map'])
    else:
        gdf[column] = np.nan

    # populate classification_str based on classification
    gdf['classification_str'] = gdf['classification'].replace({
        1: "land",
        2: "land_near_water ",
        3: "water_near_land ",
        4: "open_water ",
        5: "dark_water  ",
        6: "low_coh_water_near_land  ",
        7: "open_low_coh_water "
    })

# only keep the rows where classification is not 1 (land)
gdf = gdf[gdf['classification'] != 1]

# print all gdf types
print(gdf.dtypes)

# Save the GeoDataFrame to a shapefile
output_dir = Path(config['output_dir'])
output_dir.mkdir(exist_ok=True)
output_file = output_dir / f'water_mask_{datetime.now().strftime("%Y%m%d%H%M%S")}.geotiff'

# sloppy fix for index64 pandas issue
gdf["row_id"] = gdf.index + 1
gdf.reset_index(drop=True, inplace=True)
gdf.set_index("row_id", inplace = True)

# save to geotiff
gdf.to_file(output_file, driver='GPKG')


# 7. (Optional) Convert 'height' to meters above sea level if needed
#    The data originates from the SWOT mission and uses a custom reference ellipsoid.
#    For example, if you have an offset or scale factor, you would apply it here.

# 8. Plot the resulting GeoDataFrame, colored by 'height'
# fig, ax = plt.subplots(figsize=(10, 10))
# gdf.plot(column='height', ax=ax, legend=True, cmap='viridis')
# bbox_gdf.boundary.plot(ax=ax, color='black')

# Add a basemap (requires contextily)
# ctx.add_basemap(ax, crs=gdf.crs, source=ctx.providers.OpenStreetMap.Mapnik)

# plt.show()


KeyboardInterrupt: 